# CNOT-Qiskit

In [2]:
import numpy as np
import re
import glob
import csv

from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import CouplingMap


# from qiskit.test.mock import FakeProvider
# from qiskit.converters import circuit_to_dag
# from qiskit.dagcircuit import DAGCircuit
# from qiskit.providers import provider
# from qiskit.visualization import plot_circuit_layout

Define a few coupling maps:

In [6]:
tokyo_20_list = np.array([[1,2],[2,5],[5,6],[6,8],
                [20,3],[3,4],[4,7],[7,9],
                [19,16],[16,15],[15,12],[12,10],
                [18,17],[17,14],[14,13],[13,11],
                [1,20],[20,19],[19,18],
                [2,3],[3,16],[16,17],
                [5,4],[4,15],[15,14],
                [6,7],[7,12],[12,13],
                [8,9],[9,10],[10,11],
                [2,4],[5,3],[6,9],[8,7],
                [20,16],[3,19],[4,12],[7,15],
                [16,14],[15,17],[12,11],[10,13]]) - 1



sq_16_list = np.array([[1,2],[2,5],[5,6],
                [16,3],[3,4],[4,7],
                [15,12],[12, 11],[11, 8],
                [14,13],[13,10],[9,10],
                [1, 16],[16,15],[15,14],
                [2,3],[3,12],[12, 13],
                [5,4],[4,11],[11,10],
                [6,7],[7,8],[8,9]]) - 1

simple_4_list = np.array([[1,2],[2,3],[3,4],[4,1]]) - 1

simple_6_list = np.array([[1,2],[2,4],[3,4],[3,1], [3,5],[5,6],[6,4]]) - 1

tokyo_20 = CouplingMap(tokyo_20_list)
tokyo_20.make_symmetric()

sq_16 = CouplingMap(sq_16_list)
sq_16.make_symmetric()

simple_4 = CouplingMap(simple_4_list)
simple_4.make_symmetric()

simple_6 = CouplingMap(simple_6_list)
simple_6.make_symmetric()

qubit_to_cmap = {4: simple_4, 6: simple_6, 16: sq_16, 20: tokyo_20}

In [1]:
def get_circ_from_file(file):
    with open(file, 'r') as f:
        lines = f.readlines()

    num_qu = int(lines[0].strip())

    circ = QuantumCircuit(num_qu, 0)
    gate_map = {'X': circ.x, 'Y': circ.y, 'Z': circ.z,
        'H': circ.h, 'T': circ.t, 'T+': circ.tdg,
        'P': circ.s, 'P+': circ.sdg}

    for line in lines[1:]:
        line = line.strip().split(' ')
        if(line[0] == 'CNOT'):
            _, qu1, qu2 = line
            qu1, qu2 = int(qu1), int(qu2)
            circ.cnot(qu1, qu2)
        else:
            gate, qu = line
            qu = int(qu)
            gate_map[gate](qu)
    return circ

In [7]:
def my_transpile(circ, coupling_map):
    transpiled_circ = transpile(circ, backend=None, 
                                    coupling_map=coupling_map,
                                    optimization_level=0,
                                    basis_gates=['x', 'y','z','t','tdg','s','sdg','cx','h'],
                                    layout_method='trivial')
    return transpiled_circ

In [27]:
for num_qubit in (16, 20):
    out_file = open(f'out/out_{num_qubit}.csv', 'w', newline='')
    writer = csv.writer(out_file)
    writer.writerow(['Qubit', 'Initial CNOT Count', 'File', 'Synthesized CNOT Count', 'Overhead'])
    cmap = qubit_to_cmap[num_qubit]
    all_files = glob.glob(f'../cnotcount/Testing/{num_qubit}*')
    for file in all_files:
        reg = re.match(f'.*{num_qubit}CNOT(\d+)_(\d+).txt', file)
        exp_count = int(reg.group(1))
        index = int(reg.group(2))
        circ = get_circ_from_file(file)
        transpiled_circ = my_transpile(circ, cmap)
        init_count = circ.count_ops()['cx']
        assert init_count == exp_count
        transpiled_count = transpiled_circ.count_ops()['cx']
        overhead = transpiled_count / init_count
        print(f'qubit: {num_qubit}, init: {init_count}, after: {transpiled_count}, overhead: {overhead}')
        writer.writerow([num_qubit, init_count, index, transpiled_count, overhead])

    out_file.close()

qubit: 16, init: 128, after: 692, overhead: 5.40625
qubit: 16, init: 128, after: 830, overhead: 6.484375
qubit: 16, init: 128, after: 707, overhead: 5.5234375
qubit: 16, init: 128, after: 716, overhead: 5.59375
qubit: 16, init: 128, after: 704, overhead: 5.5
qubit: 16, init: 128, after: 752, overhead: 5.875
qubit: 16, init: 128, after: 731, overhead: 5.7109375
qubit: 16, init: 128, after: 770, overhead: 6.015625
qubit: 16, init: 128, after: 698, overhead: 5.453125
qubit: 16, init: 128, after: 764, overhead: 5.96875
qubit: 16, init: 16, after: 73, overhead: 4.5625
qubit: 16, init: 16, after: 94, overhead: 5.875
qubit: 16, init: 16, after: 121, overhead: 7.5625
qubit: 16, init: 16, after: 76, overhead: 4.75
qubit: 16, init: 16, after: 94, overhead: 5.875
qubit: 16, init: 16, after: 88, overhead: 5.5
qubit: 16, init: 16, after: 97, overhead: 6.0625
qubit: 16, init: 16, after: 103, overhead: 6.4375
qubit: 16, init: 16, after: 73, overhead: 4.5625
qubit: 16, init: 16, after: 94, overhead: 5

## Analysis of the worst-case overhead

Given a list of CNOTs
$$\{CNOT(q_{i,1},q_{i,2}): i=0,\dots,m\},$$
the worst-case CNOT count is given by
$$\sum_{i=1}^m6(d(q_{i,1},q_{i,2})-1).$$
The worst-case overhead is 
\begin{align*}
\frac{1}{m}\sum_{i=1}^m6(d(q_{i,1},q_{i,2})-1)
&=6\Big(\frac{1}{m}\sum_{i=1}^md(q_{i,1},q_{i,2})-1\Big) \\
&=6(\operatorname{avg}\{d(q_{i,1},q_{i,2})\}-1).
\end{align*}

If we assume the control and target qubits are randomly chosen with equal probability, then $$\operatorname{avg}\{d(q_{i,1},q_{i,2})\}=\operatorname{avg}\{d(q_k,q_l):k,l=0,\dots,N\}.$$

In fact, Qiskit provides a simple API to calculate the distance of any two qubits in an adjacency graph:

In [ ]:
print(sq_16.distance_matrix)

[[0. 1. 2. 3. 2. 3. 4. 5. 6. 5. 4. 3. 4. 3. 2. 1.]
 [1. 0. 1. 2. 1. 2. 3. 4. 5. 4. 3. 2. 3. 4. 3. 2.]
 [2. 1. 0. 1. 2. 3. 2. 3. 4. 3. 2. 1. 2. 3. 2. 1.]
 [3. 2. 1. 0. 1. 2. 1. 2. 3. 2. 1. 2. 3. 4. 3. 2.]
 [2. 1. 2. 1. 0. 1. 2. 3. 4. 3. 2. 3. 4. 5. 4. 3.]
 [3. 2. 3. 2. 1. 0. 1. 2. 3. 4. 3. 4. 5. 6. 5. 4.]
 [4. 3. 2. 1. 2. 1. 0. 1. 2. 3. 2. 3. 4. 5. 4. 3.]
 [5. 4. 3. 2. 3. 2. 1. 0. 1. 2. 1. 2. 3. 4. 3. 4.]
 [6. 5. 4. 3. 4. 3. 2. 1. 0. 1. 2. 3. 2. 3. 4. 5.]
 [5. 4. 3. 2. 3. 4. 3. 2. 1. 0. 1. 2. 1. 2. 3. 4.]
 [4. 3. 2. 1. 2. 3. 2. 1. 2. 1. 0. 1. 2. 3. 2. 3.]
 [3. 2. 1. 2. 3. 4. 3. 2. 3. 2. 1. 0. 1. 2. 1. 2.]
 [4. 3. 2. 3. 4. 5. 4. 3. 2. 1. 2. 1. 0. 1. 2. 3.]
 [3. 4. 3. 4. 5. 6. 5. 4. 3. 2. 3. 2. 1. 0. 1. 2.]
 [2. 3. 2. 3. 4. 5. 4. 3. 4. 3. 2. 1. 2. 1. 0. 1.]
 [1. 2. 1. 2. 3. 4. 3. 4. 5. 4. 3. 2. 3. 2. 1. 0.]]


Therefore, the average distance is simply

In [ ]:
print(simple_4.distance_matrix.mean())
print(simple_6.distance_matrix.mean())
print(sq_16.distance_matrix.mean())
print(tokyo_20.distance_matrix.mean())

1.0
1.3888888888888888
2.5
2.14


For example, for `sq_16`, the worst case overhead $\approx 6(2.5-1)=900\%$.

Results from the previous section shows that Qiskit's built-in transpiler can consistantly reduce this overhead to $\approx 600\%$.

# A simple topology 

In [52]:
cmap = qubit_to_cmap[6]
circ = get_circ_from_file('6CNOT4_1.txt')
init_c_count = 4
transpiled_circ, cnot_count = trans(circ, cmap)
overhead = cnot_count / init_c_count
print(f'qubit: {circ.num_qubits}, init: {init_c_count}, after: {cnot_count}, overhead: {overhead}')

qubit: 6, init: 4, after: 19, overhead: 4.75


In [53]:
circ.draw()

┌─────┐┌───┐┌─────┐      ┌───┐                
q_0: ┤ Tdg ├┤ T ├┤ Tdg ├──■───┤ T ├────────────────
     └─────┘├───┤└┬───┬┘  │  ┌┴───┴┐┌───┐          
q_1: ───────┤ X ├─┤ T ├───┼──┤ Tdg ├┤ H ├──■───────
      ┌───┐ └─┬─┘┌┴───┴┐  │  └─────┘└───┘┌─┴─┐     
q_2: ─┤ X ├───┼──┤ Tdg ├──┼──────────────┤ X ├─────
      ├───┤   │  └┬───┬┘  │   ┌───┐ ┌───┐└───┘┌───┐
q_3: ─┤ T ├───■───┤ T ├───┼───┤ H ├─┤ T ├──■──┤ X ├
      └───┘       └───┘ ┌─┴─┐ └───┘ └───┘┌─┴─┐└───┘
q_4: ───────────────────┤ X ├────────────┤ X ├─────
                        └───┘            └───┘     
q_5: ──────────────────────────────────────────────

In [54]:
transpiled_circ.draw()

┌─────┐       ┌───┐          ┌───┐ ┌───┐ ┌─────┐┌───┐┌───┐          »
q_0 -> 0 ┤ Tdg ├───■───┤ X ├──■───────┤ X ├─┤ T ├─┤ Tdg ├┤ H ├┤ X ├───────■──»
         └─────┘ ┌─┴─┐ └─┬─┘┌─┴─┐┌───┐└─┬─┘┌┴───┴┐└─────┘├───┤└─┬─┘       │  »
q_1 -> 1 ────────┤ X ├───■──┤ X ├┤ T ├──┼──┤ Tdg ├───■───┤ T ├──┼────■────┼──»
          ┌───┐ ┌┴───┴┐     ├───┤└───┘  │  └─────┘ ┌─┴─┐ └───┘  │  ┌─┴─┐  │  »
q_2 -> 2 ─┤ X ├─┤ Tdg ├──■──┤ X ├──■────┼──────────┤ X ├────────┼──┤ X ├──┼──»
          ├───┤ └─────┘  │  └─┬─┘  │    │   ┌───┐  ├───┤ ┌───┐  │  └───┘┌─┴─┐»
q_3 -> 3 ─┤ T ├──────────┼────┼────┼────■───┤ T ├──┤ H ├─┤ T ├──■───────┤ X ├»
          └───┘        ┌─┴─┐  │  ┌─┴─┐      ├───┤  └───┘ └───┘          └───┘»
q_4 -> 4 ──────────────┤ X ├──■──┤ X ├──■───┤ X ├────■───────────────────────»
                       └───┘     └───┘┌─┴─┐ └─┬─┘  ┌─┴─┐                     »
q_5 -> 5 ─────────────────────────────┤ X ├───■────┤ X ├─────────────────────»
                                      └───┘        └───┘                     »
«              ┌───┐          ┌───┐
«q_0 -> 0 ─────┤ X ├───────■──┤ X ├
«         ┌───┐└─┬─┘     ┌─┴─┐└───┘
«q_1 -> 1 ┤ X ├──┼────■──┤ X ├─────
«         └─┬─┘  │  ┌─┴─┐└───┘     
«q_2 -> 2 ──■────┼──┤ X ├──────────
«                │  └───┘          
«q_3 -> 3 ───────■────■────────────
«                     │            
«q_4 -> 4 ────────────┼────────────
«                   ┌─┴─┐          
«q_5 -> 5 ──────────┤ X ├──────────
«                   └───┘